In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
from keras.optimizers import RMSprop
import numpy as np
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback

Using TensorFlow backend.


In [4]:
# opening the file
fin = open("metadata.txt", "rb")

# spliting the 
lines = []
for line in fin:
    line = line.strip().lower()
    line = line.decode("ascii", "ignore")
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)

chars = set([c for c in text])
chars

nb_chars = len(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [5]:


model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [20]:
from keras.callbacks import LambdaCallback
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [16]:
import random, sys

In [9]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [10]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [15]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

Epoch 1/5
132693/132693 [==============================] - 928s 7ms/step - loss: 2.3345

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "and procure us some. we must all unite i"


NameError: name 'sys' is not defined

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
print(generate_text(500, 0.2)

## Generating text using a word based model

In [2]:
import string
def load_doc(filename):
    # To open the file as read only
    file = open(filename, 'rb')
    #To read all the text
    text = file.read()
    # replace '--' with a space ' '
    #for i in text:
     #   text = text.replace('--', ' ')
    # split into tokens by white space
    #for i in text:
     #   if '--' in i:
     #       text = text.replace('--', ' ')
    tokens = text.split()
    
    #To close the file
    # remove punctuation from each token
    li = list('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
    
    for i in li:
    #tokens = [se.replace(i, "") for se, i in enumerate(tokens, li) if i in se]
        if i in tokens:
            tokens = tokens.replace(i, "")
    #tokens = text.strip().lower()
    #table = str.maketrans('', '', string.punctuation)
    #print(table)
    #tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    file.close()
    return tokens

tokens = load_doc('metadata.txt')
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

[b'the', b'tempest', b'had', b'raged', b'for', b'six', b'and', b'on', b'the', b'seventh', b'seemed', b'to', b'the', b'ship', b'had', b'been', b'so', b'far', b'driven', b'from', b'its', b'that', b'no', b'one', b'on', b'board', b'knew', b'where', b'we', b'every', b'one', b'was', b'exhausted', b'with', b'fatigue', b'and', b'the', b'shattered', b'vessel', b'began', b'to', b'leak', b'in', b'many', b'the', b'oaths', b'of', b'the', b'sailors', b'were', b'changed', b'to', b'and', b'each', b'thought', b'only', b'how', b'to', b'save', b'his', b'own', b'said', b'to', b'my', b'terrified', b'who', b'were', b'clinging', b'round', b'can', b'save', b'us', b'if', b'he', b'to', b'him', b'nothing', b'is', b'but', b'if', b'he', b'thinks', b'it', b'good', b'to', b'call', b'us', b'to', b'let', b'us', b'not', b'we', b'shall', b'not', b'be', b'my', b'excellent', b'wife', b'dried', b'her', b'and', b'from', b'that', b'moment', b'became', b'more', b'we', b'knelt', b'down', b'to', b'pray', b'for', b'the', b'help'

In [3]:
# organize into sequences of tokens
length = 50 + 1
sequences = []
tokens = [i.decode("utf-8") for i in tokens]
#tokens = tokens.decode("utf-8")
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 60972


In [4]:
chars = set([c for c in tokens])
chars

nb_chars = len(chars)
character_to_index = dict((c, i) for i, c in enumerate(chars))
character_to_index

index_to_character = dict((i, c) for i, c in enumerate(chars))
index_to_character

sentences = sequences
word = tokens

In [6]:
maxlen = 51
step = 1

lis = []
for i in sequences:
    lis.append(i.split())


x = np.zeros((len(sequences), maxlen, nb_chars), dtype=np.bool)
y = np.zeros((len(sequences), nb_chars), dtype=np.bool)
for i, sentence in enumerate(lis):
    for t, char in enumerate(sentence):
        x[i, t, character_to_index[char]] = 1
    y[i, character_to_index[word[i]]] = 1

MemoryError: Unable to allocate 14.6 GiB for an array with shape (60972, 51, 5029) and data type bool

In [ ]:

from keras.layers import LSTM
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = index_to_character[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [ ]:
import random, sys

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "weights1.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

In [7]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
import numpy as np

In [8]:
import string
def load_doc(filename):
    # To open the file as read only
    file = open(filename, 'rb')
    #To read all the text
    text = file.read()
    # replace '--' with a space ' '
    #for i in text:
     #   text = text.replace('--', ' ')
    # split into tokens by white space
    #for i in text:
     #   if '--' in i:
     #       text = text.replace('--', ' ')
    tokens = text.split()
    
    #To close the file
    # remove punctuation from each token
    li = list('!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
    
    for i in li:
    #tokens = [se.replace(i, "") for se, i in enumerate(tokens, li) if i in se]
        if i in tokens:
            tokens = tokens.replace(i, "")
    #tokens = text.strip().lower()
    #table = str.maketrans('', '', string.punctuation)
    #print(table)
    #tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    file.close()
    return tokens

tokens = load_doc('metadata.txt')
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = 50 + 1
sequences = []
tokens = [i.decode("utf-8") for i in tokens]
#tokens = tokens.decode("utf-8")
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))



[b'the', b'tempest', b'had', b'raged', b'for', b'six', b'and', b'on', b'the', b'seventh', b'seemed', b'to', b'the', b'ship', b'had', b'been', b'so', b'far', b'driven', b'from', b'its', b'that', b'no', b'one', b'on', b'board', b'knew', b'where', b'we', b'every', b'one', b'was', b'exhausted', b'with', b'fatigue', b'and', b'the', b'shattered', b'vessel', b'began', b'to', b'leak', b'in', b'many', b'the', b'oaths', b'of', b'the', b'sailors', b'were', b'changed', b'to', b'and', b'each', b'thought', b'only', b'how', b'to', b'save', b'his', b'own', b'said', b'to', b'my', b'terrified', b'who', b'were', b'clinging', b'round', b'can', b'save', b'us', b'if', b'he', b'to', b'him', b'nothing', b'is', b'but', b'if', b'he', b'thinks', b'it', b'good', b'to', b'call', b'us', b'to', b'let', b'us', b'not', b'we', b'shall', b'not', b'be', b'my', b'excellent', b'wife', b'dried', b'her', b'and', b'from', b'that', b'moment', b'became', b'more', b'we', b'knelt', b'down', b'to', b'pray', b'for', b'the', b'help'